In [ ]:
!pip install -qq PyPDF2
import PyPDF2
import re
from nltk.corpus import words
from nltk.tokenize import sent_tokenize, word_tokenize, wordpunct_tokenize
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from tqdm import tqdm
import glob
import numpy as np
import nltk
import re
import pandas as pd
import subprocess
import math
import matplotlib.pyplot as plt
nltk.download("words")
!pip install nltk
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download("words")

swords = stopwords.words('english')
swords.append("the")
swords.append("we")
swords.append("this")

In [ ]:
#read in training data as pdfs
import os
import pandas as pd

fileNames = []
numPages = []
dirs = ["/content/2017/","/content/2018/","/content/2019/","/content/2020/"]

for ss in dirs:

  for filename in os.listdir(ss):
     
          dir = ss+ filename
          pdffileobj=open(dir,'rb')
          pdfreader=PyPDF2.PdfFileReader(pdffileobj)
          pageNums = pdfreader.numPages
          fileNames.append(filename[:-4])
          numPages.append(pageNums)

zipped = list(zip(fileNames, numPages))
featDF = pd.DataFrame(zipped, columns=['Name', 'numPages'])

In [ ]:
#marks csv
marks = pd.read_csv('/content/docs/ml4ed_marks.2022-08-08.csv')

In [ ]:
def getMark(markDF,reportID):
  index = markDF.loc[markDF['report_id'] == reportID]
  index = index.mark
  return index.item()

#append Student Marks
sortedMarks = []
for row in featDF.iterrows():
  pdfName = row[1].Name
  pdfName = int(pdfName)
  m = getMark(marks,pdfName)
  sortedMarks.append(m)
featDF['Mark'] = sortedMarks

In [ ]:
#read in validation data
valFileNames = pd.read_csv('/content/docs/split_val.2022-08-08.txt',header = None)
valFileNames.columns = ["fileName"]
valFileNames = valFileNames.fileName.tolist()
valMarks = []
valFiles = []
valPageNums = []
for filename in os.listdir('/content/2021/'):
  if int(filename[:-4]) in valFileNames:
    dir = ""
    m = getMark(marks,int(filename[:-4]))
    valFiles.append(filename[:-4])
    valMarks.append(m)
    dir = '/content/2021/' + filename
    pdffileobj=open(dir,'rb')
    pdfreader=PyPDF2.PdfFileReader(pdffileobj)
    valPageNums.append(pdfreader.numPages)

zipped = list(zip(valFiles, valMarks,valPageNums))
val_feat_df = pd.DataFrame(zipped, columns=['Name', 'Mark',"numPages"])

#read in test data
valFileNames = pd.read_csv('/content/docs/split_val.2022-08-08.txt',header = None)
valFileNames.columns = ["fileName"]
valFileNames = valFileNames.fileName.tolist()
valMarks = []
valFiles = []
valPageNums = []
for filename in os.listdir('/content/2021/'):
  if int(filename[:-4]) not in valFileNames:
    dir = ""
    m = getMark(marks,int(filename[:-4]))
    valFiles.append(filename[:-4])
    valMarks.append(m)
    dir = '/content/2021/' + filename
    pdffileobj=open(dir,'rb')
    pdfreader=PyPDF2.PdfFileReader(pdffileobj)
    valPageNums.append(pdfreader.numPages)

zipped = list(zip(valFiles, valMarks,valPageNums))
test_feat_df = pd.DataFrame(zipped, columns=['Name', 'Mark',"numPages"])

Additional Features

In [ ]:
#unzip text only files
!unzip -qq /content/20170005_00001.zip
!unzip -qq /content/20180028_00001.zip
!unzip -qq /content/20190015_00001.zip
!unzip -qq /content/20200006_00001.zip
!unzip -qq /content/20210019_00001.zip

In [ ]:
def getPageNums_and_Mark(DF, reportID):
  index = DF.loc[DF['Name'] == reportID]
  m = index.Mark
  p = index.numPages
  return m.item(), p.item()

def get_txt_feats(txt_fn):
    with open(txt_fn) as f:
        lines = f.readlines()
        # text = 

    n_words = 0
    spell_mistakes = []
    for word in wordpunct_tokenize("".join(lines)):
        if not word.isalpha():
            continue
        n_words += 1
        if not (word.lower() in en_words):
            spell_mistakes.append(word)

    n_figures = 0
    n_tables = 0
    n_equations = 0
    n_references = 0
    n_lines_start_at = 0
    for line in lines:
        # if re.match("^figure.*", line.lower()):
        if re.match(r"^figure.*[^\d]$", line.strip().lower()):
            # print("Fig:", line)
            n_figures += 1
        # elif re.match("^table.*", line.strip().lower()):
        elif re.match("^table.*[^\d]$", line.strip().lower()):
            # print("Tab:", line)
            n_tables += 1
        elif re.match("^\([\d\.]*\)$", line.strip().lower()):
            # print("Eq:", line)
            n_equations += 1
        elif re.match(r"^\[\d*\]", line.lower()):
            n_references += 1
        elif re.match(r"^\@.*", line.lower()):
            n_lines_start_at += 1

    return {
        "n_words": n_words,
        "spell_mistakes": spell_mistakes,
        "n_figures": n_figures,
        "n_tables": n_tables,
        "n_equations": n_equations,
        "n_references": n_references,
        "n_lines_start_at": n_lines_start_at
        }

In [ ]:
# extract training data features
n_words_List = []
spell_mistakes_List = []
spell_mistake_types_list = []
figure_list = []
tables_list = []
equation_list = []
n_ref_list = []
n_lines_start_list = []
projectNames = []
projectMarks = []
pageCounts = []

for filename in os.listdir('/content/HERMAN_TEXT'):
  if filename[0] != "s":
    if filename.endswith('txt') and int(filename[:4]) != 2021:
      
      name = '/content/HERMAN_TEXT/'+ filename
      # n_words, spell_mistakes, n_figures, n_tables, n_equations, n_references, n_lines_start_at = get_txt_feats(filename)
      features = get_txt_feats(name)
      n_words_List.append(features['n_words'])
      spell_mistakes_List.append(len(features["spell_mistakes"]))
      spell_mistake_types_list.append(len(set(features["spell_mistakes"])))
      figure_list.append(features["n_figures"])
      tables_list.append(features["n_tables"])
      equation_list.append(features["n_equations"])
      n_ref_list.append(features["n_references"])
      n_lines_start_list.append(features["n_lines_start_at"])
      projectNames.append(filename[:8])
      p_mark, p_count = getPageNums_and_Mark(featDF, filename[:8])
      projectMarks.append(p_mark)
      pageCounts.append(p_count)
    
zipped = list(zip(projectNames,n_words_List, pageCounts, spell_mistakes_List,spell_mistake_types_list,figure_list,tables_list,equation_list,n_ref_list, n_lines_start_list,projectMarks))
feature_train_df = pd.DataFrame(zipped, columns=['reportID', 'numWords',"numPages","n_spell_mistakes",'n_spell_mistakes_types',"n_figures","n_tables","n_equations","n_references","n_lines_start","mark"])

# extract features from validation data

n_words_List = []
spell_mistakes_List = []
spell_mistake_types_list = []
figure_list = []
tables_list = []
equation_list = []
n_ref_list = []
n_lines_start_list = []
projectNames = []
projectMarks = []
pageCounts = []

for filename in os.listdir('/content/HERMAN_TEXT'):
  if filename[0] != "s":
    if filename.endswith('txt') and int(filename[:4]) == 2021 and int(filename[5:8])<=59:

      name = '/content/HERMAN_TEXT/'+ filename
      # n_words, spell_mistakes, n_figures, n_tables, n_equations, n_references, n_lines_start_at = get_txt_feats(filename)
      features = get_txt_feats(name)
      n_words_List.append(features['n_words'])
      spell_mistakes_List.append(len(features["spell_mistakes"]))
      spell_mistake_types_list.append(len(set(features["spell_mistakes"])))
      figure_list.append(features["n_figures"])
      tables_list.append(features["n_tables"])
      equation_list.append(features["n_equations"])
      n_ref_list.append(features["n_references"])
      n_lines_start_list.append(features["n_lines_start_at"])
      projectNames.append(filename[:8])
      p_mark, p_count = getPageNums_and_Mark(val_feat_df, filename[:8])
      projectMarks.append(p_mark)
      pageCounts.append(p_count)


zipped = list(zip(projectNames,n_words_List, pageCounts, spell_mistakes_List,spell_mistake_types_list,figure_list,tables_list,equation_list,n_ref_list, n_lines_start_list,projectMarks))
feature_val_df = pd.DataFrame(zipped, columns=['reportID', 'numWords',"numPages","n_spell_mistakes",'n_spell_mistakes_types',"n_figures","n_tables","n_equations","n_references","n_lines_start","mark"])

# extract features from test data

n_words_List = []
spell_mistakes_List = []
spell_mistake_types_list = []
figure_list = []
tables_list = []
equation_list = []
n_ref_list = []
n_lines_start_list = []
projectNames = []
projectMarks = []
pageCounts = []

for filename in os.listdir('/content/HERMAN_TEXT'):
  if filename[0] != "s":
    if filename.endswith('txt') and int(filename[:4]) == 2021 and int(filename[5:8])>59:

      name = '/content/HERMAN_TEXT/'+ filename
      # n_words, spell_mistakes, n_figures, n_tables, n_equations, n_references, n_lines_start_at = get_txt_feats(filename)
      features = get_txt_feats(name)
      n_words_List.append(features['n_words'])
      spell_mistakes_List.append(len(features["spell_mistakes"]))
      spell_mistake_types_list.append(len(set(features["spell_mistakes"])))
      figure_list.append(features["n_figures"])
      tables_list.append(features["n_tables"])
      equation_list.append(features["n_equations"])
      n_ref_list.append(features["n_references"])
      n_lines_start_list.append(features["n_lines_start_at"])
      projectNames.append(filename[:8])
      p_mark, p_count = getPageNums_and_Mark(test_feat_df, filename[:8])
      projectMarks.append(p_mark)
      pageCounts.append(p_count)
      
zipped = list(zip(projectNames,n_words_List, pageCounts, spell_mistakes_List,spell_mistake_types_list,figure_list,tables_list,equation_list,n_ref_list, n_lines_start_list,projectMarks))
feature_test_df = pd.DataFrame(zipped, columns=['reportID', 'numWords',"numPages","n_spell_mistakes",'n_spell_mistakes_types',"n_figures","n_tables","n_equations","n_references","n_lines_start","mark"])


File size

In [ ]:
fileSize = []
fileMark = []
trainNames = []
dirs = ["/content/2017/","/content/2018/","/content/2019/","/content/2020/"]

for ss in dirs:

  for filename in os.listdir(ss):
     
          dir = ss+ filename
          filename = dir
          file_stats = os.stat(filename)
          fileSize.append(file_stats.st_size / (1024 * 1024))
          m = getMark(marks,int(filename[14:-4]))
          fileMark.append(m)
          trainNames.append(int(filename[14:-4]))


valFileNames = pd.read_csv('/content/docs/split_val.2022-08-08.txt',header = None)
valFileNames.columns = ["fileName"]
valFileNames = valFileNames.fileName.tolist()
valMarks = []
valFileSize = []
valNames = []

for filename in os.listdir('/content/2021/'):
  if int(filename[:-4]) in valFileNames:
    dir = ""
    m = getMark(marks,int(filename[:-4]))
    valMarks.append(m)
    dir = '/content/2021/' + filename
    filename = dir
    file_stats = os.stat(filename)
    valFileSize.append(file_stats.st_size / (1024 * 1024))
    valNames.append(int(filename[14:-4]))


In [ ]:
valFileNames = pd.read_csv('/content/docs/split_val.2022-08-08.txt',header = None)
valFileNames.columns = ["fileName"]
valFileNames = valFileNames.fileName.tolist()
valMarks = []
valFileSize = []
valNames = []

for filename in os.listdir('/content/2021/'):
  if int(filename[:-4]) not in valFileNames:
    dir = ""
    m = getMark(marks,int(filename[:-4]))
    valMarks.append(m)
    dir = '/content/2021/' + filename
    filename = dir
    file_stats = os.stat(filename)
    valFileSize.append(file_stats.st_size / (1024 * 1024))
    valNames.append(int(filename[14:-4]))

zipped = list(zip(valNames, valFileSize))
featDF = pd.DataFrame(zipped, columns=['Name', 'fileSize'])
featDF.to_csv("test_FILESIZE.csv")

In [ ]:
zipped = list(zip(trainNames, fileSize))
featDF = pd.DataFrame(zipped, columns=['Name', 'fileSize'])
featDF.to_csv("train_FILESIZE.csv")

zipped = list(zip(valNames, valFileSize))
featDF = pd.DataFrame(zipped, columns=['Name', 'fileSize'])
featDF.to_csv("val_FILESIZE.csv")

Word Count

In [ ]:

analysys_tokens = []
trainText = []
trainLabels = []
trainMarks = []

#text files directory
for filename in os.listdir('/content/HERMAN_TEXT'):
  if filename[0] != "s":
    if filename.endswith('txt') and int(filename[:4]) != 2021 and int(filename[:8])!=20170049:
      
      name = '/content/HERMAN_TEXT/'+ filename
      with open(name) as f:
        lines = f.read()

      lines = lines.lower()
      text_tokens = word_tokenize(lines)
      tokens_without_sw = [word for word in text_tokens if not word in swords]
      analysys_tokens.append(tokens_without_sw)
      text = (" ").join(tokens_without_sw)
      trainText.append(text)
      trainLabels.append(filename[:8])
      trainMarks.append(getMark(df_concat,int(filename[:8])))
      

      
valText = []
valLabels = []  
valMarks = []

for filename in os.listdir('/content/HERMAN_TEXT'):
  if filename[0] != "s":
    if filename.endswith('txt') and int(filename[:4]) == 2021 and int(filename[5:8])<=59:

      name = '/content/HERMAN_TEXT/'+ filename

      with open(name) as f:
        lines = f.read()

      lines = lines.lower()
      text_tokens = word_tokenize(lines)
      tokens_without_sw = [word for word in text_tokens if not word in swords]
      text = (" ").join(tokens_without_sw)
      valText.append(text)
      valLabels.append(filename[:8])
      valMarks.append(getMark(df_concat,int(filename[:8])))

testText = []
testLabels = []  

for filename in os.listdir('/content/HERMAN_TEXT'):
  if filename[0] != "s":
    if filename.endswith('txt') and int(filename[:4]) == 2021 and int(filename[5:8]) > 59:

      name = '/content/HERMAN_TEXT/'+ filename

      with open(name) as f:
        lines = f.read()

      lines = lines.lower()
      text_tokens = word_tokenize(lines)
      tokens_without_sw = [word for word in text_tokens if not word in swords]
      text = (" ").join(tokens_without_sw)
      testText.append(text)
      testLabels.append(filename[:8])

In [ ]:
# preprocess

def preProcess(training_sentences):
  
  for i in range(len(training_sentences)):
    training_sentences[i] = training_sentences[i].lower()
    training_sentences[i] = re.sub("-", '', training_sentences[i])
    training_sentences[i] = re.sub(",", '', training_sentences[i])
    training_sentences[i] = re.sub(r"\(", '', training_sentences[i])
    training_sentences[i] = re.sub(r"\)", '', training_sentences[i])
    training_sentences[i] = re.sub(";", '', training_sentences[i])
    training_sentences[i] = re.sub(":", '', training_sentences[i])
    training_sentences[i] = re.sub(r'\\', ' ', training_sentences[i])
    training_sentences[i] = re.sub(r"[^a-z\s]", '', training_sentences[i])

  return training_sentences

In [ ]:
trainText = preProcess(trainText)
valText = preProcess(valText)
testText = preProcess(testText)

In [ ]:
#extract vocubulary
def extract_words(data):
  V = {}
  
  for i in range(len(data)):
    words = data[i].split()
    
    for j in words:
      if j in V:
        V[j] +=1
      else:
        V[j] = 1

  return V

def sortFreqDict(freqdict):
    aux = [(freqdict[key], key) for key in freqdict]
    aux.sort()
    aux.reverse()
    return aux

In [ ]:
V_train = extract_words(trainText)
V_train_sorted = sortFreqDict(V_train)

In [ ]:
top_k_words = 10
count_words = []
for k in range(top_k_words):
  # if len(V_train_sorted[k][1]) >1:
    count_words.append(V_train_sorted[k][1])

In [ ]:
def countWords(top_k_words, text):
  wCounts = []
  for j in range(len(top_k_words)):
    count = 0
    
    words = text.split()

    for k in words:
      if k == top_k_words[j]:
        count += 1

    wCounts.append(count)
  return wCounts

In [ ]:
counts_train = []

for i in range(len(trainText)):

  top_counts = countWords(count_words, trainText[i])
  counts_train.append(top_counts)

counts_val = []

for i in range(len(valText)):

  top_counts = countWords(count_words, valText[i])
  counts_val.append(top_counts)


counts_test = []

for i in range(len(testText)):

  top_counts = countWords(count_words, testText[i])
  counts_test.append(top_counts)

In [ ]:
col_names = []
for i in range(10):
  name = "count_" + str(i)
  col_names.append(name)

In [ ]:
zipped = list(zip(trainLabels, counts_train))
featDF_train = pd.DataFrame(zipped, columns=['report_id', 'counts'])
m = pd.DataFrame(featDF_train["counts"].to_list(), columns=col_names)
featDF_train = pd.concat([featDF_train["report_id"],m],axis = 1)


zipped = list(zip(valLabels, counts_val))
featDF_val = pd.DataFrame(zipped, columns=['report_id', 'counts'])
m = pd.DataFrame(featDF_val["counts"].to_list(), columns=col_names)
featDF_val = pd.concat([featDF_val["report_id"],m],axis = 1)

zipped = list(zip(testLabels, counts_val))
featDF_test = pd.DataFrame(zipped, columns=['report_id', 'counts'])
m = pd.DataFrame(featDF_test["counts"].to_list(), columns=col_names)
featDF_test = pd.concat([featDF_test["report_id"],m],axis = 1)

BERTOPIC

In [ ]:
swords = stopwords.words('english')
swords.append("the")
swords.append("we")
swords.append("this")
swords.append("project")
swords.append("report")
swords.append("using")

In [ ]:
#ABSTRACT ONLY TEXT
AbstractText = []
id_names = []
years = []
#TRAIN
for filename in os.listdir("/content/EEabstracts/"):

  file = '/content/EEabstracts/' + filename
  if filename[:4] != "2021":
    with open(file) as f:
          lines = f.read()
    f.close()
    
    lines= lines.lower()
    text_tokens = word_tokenize(lines)
    tokens_without_sw = [word for word in text_tokens if not word in swords]
    text = (" ").join(tokens_without_sw)
    years.append(filename[:4])
    AbstractText.append(text)
    id_names.append(filename[:8])


zipped = list(zip(id_names,AbstractText, years))
summaryDF = pd.DataFrame(zipped, columns=['reportID', "sumText","year"])

#VALIDATION
AbstractText = []
id_names = []

for filename in os.listdir("/content/EEabstracts/"):

  file = '/content/EEabstracts/' + filename
  if int(filename[:8]) <= 20210059 and filename[:4] == "2021":
    with open(file) as f:
          lines = f.read()
    f.close()
    lines = lines.lower()
    text_tokens = word_tokenize(lines)
    tokens_without_sw = [word for word in text_tokens if not word in swords]
    text = (" ").join(tokens_without_sw)

    AbstractText.append(text)
    id_names.append(filename[:8])

zipped = list(zip(id_names,AbstractText))
summaryDFVAL = pd.DataFrame(zipped, columns=['reportID', "sumText"])

#TEST
AbstractText = []
id_names = []

for filename in os.listdir("/content/EEabstracts/"):

  file = '/content/EEabstracts/' + filename
  if int(filename[:8]) > 20210059 and filename[:4] == "2021":
    with open(file) as f:
          lines = f.read()
    f.close()
    lines = lines.lower()
    text_tokens = word_tokenize(lines)
    tokens_without_sw = [word for word in text_tokens if not word in swords]
    text = (" ").join(tokens_without_sw)

    AbstractText.append(text)
    id_names.append(filename[:8])

zipped = list(zip(id_names,AbstractText))
summaryDFTEST = pd.DataFrame(zipped, columns=['reportID', "sumText"])

In [ ]:
!pip install -qq bertopic
!pip install -qq bertopic[visualization]
from bertopic import BERTopic
!pip install -U kaleido
from sklearn.cluster import KMeans

In [ ]:

cluster_model = KMeans(n_clusters = 8)
model = BERTopic(verbose=True,nr_topics = 8,language = "English",hdbscan_model = cluster_model)
 
#convert to list 
docs = summaryDF.sumText.to_list()
dates = summaryDF.year.to_list()
 
topics, probabilities = model.fit_transform(docs)

In [ ]:
topics_over_time = model.topics_over_time(docs, dates)
c = model.visualize_topics_over_time(topics_over_time)
# c.update_layout(title = "")
c.update_xaxes(nticks = 4)
c.write_image("/content/topicsOverTime.pdf")
c.show()

In [ ]:
topic_labels = model.generate_topic_labels(nr_words=3, topic_prefix=True, word_length=None, separator='_')

In [ ]:
topic_labels

In [ ]:
model.visualize_barchart(n_words = 5)

In [ ]:
#get val data topics
valTopics = []

for i in summaryDFVAL.sumText:
  new_doc = i
  t = model.transform([new_doc])
  valTopics.append(t[0][0])

#get test data topics
testTopics = []

for i in summaryDFTEST.sumText:
  new_doc = i
  t = model.transform([new_doc])
  testTopics.append(t[0][0])

summaryDF["topic"] = topics

In [ ]:
ids = summaryDF.reportID.tolist()
id_marks = []
for i in range(len(ids)):

  iv_mark = getMark(marks,int(ids[i]))
  id_marks.append(iv_mark)

summaryDF["mark"] = id_marks

ids = summaryDFVAL.reportID.tolist()
id_marks = []
for i in range(len(ids)):

  iv_mark = getMark(marks,int(ids[i]))
  id_marks.append(iv_mark)

summaryDFVAL["mark"] = id_marks

ids = summaryDFTEST.reportID.tolist()
id_marks = []
for i in range(len(ids)):

  iv_mark = getMark(marks,int(ids[i]))
  id_marks.append(iv_mark)

summaryDFTEST["mark"] = id_marks

In [ ]:
def returnTopic(df,reportID):
  index = df[df['reportID'] == reportID].topic.tolist()
  index = index[0] 

  return index